In [3]:
import pandas as pd
from premise import NewDatabase
from brightway2 import projects, databases, Database
from pyam import IamDataFrame

# CONFIGURATION 
CSV_PATH = "out/DE_GRIDMIX_SCENARIO_02_PREDICTIONS.csv"
PROJECT_NAME = "DE_GRIDMIX"
BIOSPHERE_NAME = "biosphere3"
SOURCE_DB_NAME = "TEST_DB"

# Add the hydron / waste generation technologies to the database
CUSTOM_SCENARIOS = pd.DataFrame([
    {
        "model": "custom",
        "pathway": "Base",
        "variable": "Electricity|Production|Hydrogen",
        "name": "market group for hydrogen",
        "unit": "kilowatt hour",
        "location": "DE"
    },
    {
        "model": "custom",
        "pathway": "Base",
        "variable": "Electricity|Production|Waste",
        "name": "electricity production from municipal solid waste, incineration",
        "unit": "kilowatt hour",
        "location": "DE"
    }
])

# Set or create Brightway project
if PROJECT_NAME not in projects:
    projects.set_current(PROJECT_NAME)
else:
    projects.set_current(PROJECT_NAME)

# Ensure biosphere3 is available
if BIOSPHERE_NAME not in databases:
    from bw2io import create_default_biosphere3
    create_default_biosphere3()




# === Ensure TEST_DB exists with dummy content ===
if SOURCE_DB_NAME not in databases:
    db = Database(SOURCE_DB_NAME)

    def safe_str(val):
        return str(val).strip() if val else ""

    def make_dataset(name, ref_product, code):
        return {
            'name': safe_str(name),
            'reference product': safe_str(ref_product),
            'unit': safe_str('kilowatt hour'),
            'location': safe_str('DE'),
            'type': 'process',
            'exchanges': [
                {
                    'input': (SOURCE_DB_NAME, code),
                    'name': safe_str("dummy input"),
                    'reference product': safe_str("dummy input"),
                    'amount': 1,
                    'unit': safe_str('kilowatt hour'),
                    'location': safe_str('DE'),
                    'type': 'technosphere',
                }
            ]
        }


    dummy_input = {
        'name': safe_str('dummy input'),
        'reference product': safe_str('dummy input'),
        'unit': safe_str('kilowatt hour'),
        'location': safe_str('DE'),
        'type': 'process',
        'exchanges': []
    }

    db.write({
        (SOURCE_DB_NAME, "dummy_input"): dummy_input,
        (SOURCE_DB_NAME, "dummy_hydrogen"): make_dataset("market group for hydrogen", "hydrogen", "dummy_input"),
        (SOURCE_DB_NAME, "dummy_waste"): make_dataset("electricity production from municipal solid waste, incineration", "electricity", "dummy_input"),
    })


# Load scenario data 
df = pd.read_csv(CSV_PATH)
# convert to Iam Dataframe
iam_df = IamDataFrame(df)


# Init Database
db = NewDatabase(
    scenarios=[{
        "model": "remind",
        "pathway": "SSP2-Base",
        "year": 2030,
        "iam_data": iam_df
    }],
    source_db=SOURCE_DB_NAME,
    biosphere_name="biosphere3",
    external_scenarios=[CUSTOM_SCENARIOS]
)


Reading unencrypted IAM output files.
premise v.(2, 2, 7)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+--------------------------------------------

100%|██████████| 2/2 [00:00<?, ?it/s]


Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


100%|██████████| 2/2 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'strip'